# **CNN SCRATCH**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import random
import glob as gb
from collections import Counter
from sklearn.metrics import classification_report

from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,precision_score, recall_score, f1_score


In [ ]:
width = 224
height = 224
color_channels = 3
image_size = (width, height, color_channels)
num_classes = 4

train_path = '/kaggle/input/braintumor/Training'
test_path = '/kaggle/input/braintumor/Testing'



In [ ]:
def read_images_from_directory(folder_path, desc):

    images = []
    labels=[]

    # Iterate through each folder in the given directory with a progress bar
    i = 0
    for folder in os.listdir(folder_path):
        files = gb.glob(pathname=str(folder_path + '/' + folder + '/*.jpg'))

        # For each file in the current folder, read the image and append its size
        for file in tqdm(files, desc=desc+" in "+folder):
            image = plt.imread(file)
            img_resized=cv2.resize(image,(width,height))
            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            images.append(img_rgb)
            labels.append(folder)



    # Count the occurrence of each unique size
    return images,labels


In [ ]:
x_train,y_train= read_images_from_directory(train_path, "Loading Training Data")

In [ ]:
x_test,y_test= read_images_from_directory(test_path, "Loading Testing Data")

In [ ]:
def visualize_data(list_of_images,labels):
  plt.figure(figsize=(20,20))
  for n,i in enumerate(list(np.random.randint(0,len(list_of_images),36))):
      plt.subplot(6,6,n+1)
      plt.imshow(list_of_images[i])
      plt.axis('off')
      plt.title(labels[i])

In [ ]:
visualize_data(x_train,y_train)

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

print(f'X_train shape  is {x_train.shape}')
print(f'X_test shape  is {x_test.shape}')
print(f'y_train shape  is {y_train.shape}')
print(f'y_test shape  is {y_test.shape}')


In [ ]:
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)



In [ ]:
y_train_encoded = to_categorical(y_train_encoded)
y_test_encoded = to_categorical(y_test_encoded)

print(y_train_encoded.shape)
print(y_test_encoded.shape)



In [ ]:
def create_custom_cnn(image_size, num_classes):
    model = tf.keras.Sequential()

    # First convolutional layer
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=image_size))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))  # Adding dropout

    # Second convolutional layer
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))  # Adding dropout

    model.add(layers.Flatten())

    # Dense layers for classification
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))  # Adding dropout
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Print model summary
    model.summary()

    return model

In [ ]:
single_model = create_custom_cnn(image_size, num_classes)

In [ ]:
history = single_model.fit(x_train, y_train_encoded, epochs=10, validation_split=0.2)

In [ ]:
test_loss, test_accuracy = single_model.evaluate(x_test, y_test_encoded)

print(f"Test Accuracy: {test_accuracy}")
print(f"Test Loss: {test_loss}")

In [ ]:

print('Training loss',np.mean(history.history['loss']))
print('Traing accuracy',np.mean(history.history['accuracy']))

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
predictions = single_model.predict(x_test)
y_pred = np.argmax(predictions, axis=1)
y_true = np.argmax(y_test_encoded, axis=1)

print(classification_report(y_true, y_pred))
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
print('Accuracy:', accuracy_score(y_true, y_pred))
# Print metrics
print('Precision:', precision)
print('Recall:', recall)
print('F1-Score:', f1)

# **VGG16**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_vgg(image_size, num_classes):
    """
    Creates, compiles, prints a summary, and visualizes a model with VGG16 as the base.

    Parameters:
    - image_size: tuple, the input size of the images (height, width, channels).
    - num_classes: int, the number of classes for the classification task.

    Returns:
    - The compiled model.
    """

    # Load pre-trained VGG16 model + higher level layers
    vgg = tf.keras.applications.VGG16(
        include_top=False,  # Exclude the top classification layer
        weights="imagenet",
        input_shape=image_size
    )

    # Freeze layers in the base model
    for layer in vgg.layers:
        layer.trainable = False

    # Create a sequential model
    model = models.Sequential()

    # Add the base model
    model.add(vgg)

    # Add a global max pooling layer
    model.add(layers.GlobalMaxPooling2D())

    # Add final classification layers
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Build the model by providing input shape
    model.build(input_shape=(None,) + image_size)

    # Print model summary
    model.summary()

    # Visualize the model
    visualization_path = 'model_visualization.png'
    tf.keras.utils.plot_model(model, to_file=visualization_path, show_shapes=True, show_layer_names=True)

    print(f"Model visualization saved to {visualization_path}")

    return model


In [ ]:
single_model =create_vgg(image_size, num_classes)

In [ ]:
history = single_model.fit(x_train, y_train_encoded, epochs=10, validation_split=0.2)


In [ ]:
test_loss, test_accuracy = single_model.evaluate(x_test, y_test_encoded)

print(f"Test Accuracy: {test_accuracy}")
print(f"Test Loss: {test_loss}")

# **Inception**

In [ ]:
# Load EfficientNetB0 base model
base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(height, width, color_channels))

# Count the total number of layers in the base model
total_layers = len(base_model.layers)

# Freeze all layers except the last 10
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Create a Sequential model
model = tf.keras.Sequential()

# Add the EfficientNetB0 base model
model.add(base_model)

# Add global pooling and flatten layers
model.add(layers.GlobalMaxPooling2D())
model.add(layers.Flatten())

# Add dense layers
model.add(layers.Dense(256, activation='relu'))

# Output layer
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    min_delta=0.001,        # Minimum change to qualify as an improvement
    patience=3,             # Number of epochs to wait for improvement
    verbose=1,              # Verbosity mode
    restore_best_weights=True # Restore the best weights after stopping
)

# Train the model
history = model.fit(
    x_train, y_train_encoded, 
    epochs=15, 
    validation_split=0.2, 
    callbacks=[early_stopping]
)

# **Res101**

In [ ]:

def create_resnet101(image_size, num_classes):
    """
    Creates, compiles, prints a summary, and visualizes a model with MobileNetV3Small and ResNet101 branches.

    Parameters:
    - image_size: tuple, the input size of the images (height, width, channels).
    - num_classes: int, the number of classes for the classification task.

    Returns:
    - The compiled model.
    """
    # Load pre-trained models
    res101 = tf.keras.applications.ResNet101(include_top=False,
                                             weights='imagenet',
                                             input_shape=image_size)

    # Freeze layers in the base model
    for layer in res101.layers:
        layer.trainable = False

    # Create a sequential model
    model = tf.keras.Sequential()

    # Add the base model
    model.add(res101)

    # Add a global max pooling layer
    model.add(layers.GlobalMaxPooling2D())

    # Add a flatten layer
    model.add(layers.Flatten())

    # Add final classification layers
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Print model summary
    model.summary()

    # Visualize the model
    visualization_path = 'model_visualization.png'
    tf.keras.utils.plot_model(model, to_file=visualization_path, show_shapes=True, show_layer_names=True)

    print(f"Model visualization saved to {visualization_path}")

    return model
